<a href="https://colab.research.google.com/github/donghyundavidchoi/Data_Analysis/blob/main/%EB%AA%A8%EB%8D%B8%EB%A7%81/%EB%AA%A8%EB%8D%B8%EB%A7%81_%EB%B9%85%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%B6%84%EC%84%9D%EA%B8%B0%EC%82%AC%EC%98%88%EC%8B%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 고객 3500명에 대한 학습용 데이터(y_train.csv, X_train.csv)를 활용하여 성별예측 모형을 만든 후, 이를 평가용 데이터(X_test.csv)에 적용하여 얻은 2482명 고객의 성별 예측값(남자일 확률)을 다음과 같은 형식의 CSV 파일로 생성하시오. (제출한 모델의 성능은 ROC-AUC 평가지표에 따라 채점

### csv파일 형식

custid,gender   
3500,0.267   
3501,0.578   
3502,9.885

# 유의사항
- 성능이 우수한 예측모형을 구축하기 위해서는 적절한 데이터 전처리, Feature Engineering, 분류 알고리즘 사용, 초매개변수 최적화, 모형 앙상블 등이 수반되어야 한다.   

- 수험번호.csv 파일이 만들어지도록 코드를 제출한다.   

- 제출한 모델의 성능은 ROC-AUC 평가지료에 따라 채점한다.   

In [4]:
!gdown --fuzzy https://drive.google.com/file/d/1UZ3j0a2oq5qFRf5NdgGnOVVypCdU3zIR/view?usp=sharing
!gdown --fuzzy https://drive.google.com/file/d/1-nbTDJjUpLSm_zBNUtJfKxT0ujWbW8IP/view?usp=sharing
!gdown --fuzzy https://drive.google.com/file/d/1pJpocRhfPvchWQcDEXba3saoAI7yDBY5/view?usp=sharing

Downloading...
From: https://drive.google.com/uc?id=1UZ3j0a2oq5qFRf5NdgGnOVVypCdU3zIR
To: /content/X_train.csv
100% 246k/246k [00:00<00:00, 80.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-nbTDJjUpLSm_zBNUtJfKxT0ujWbW8IP
To: /content/X_test.csv
100% 177k/177k [00:00<00:00, 80.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1pJpocRhfPvchWQcDEXba3saoAI7yDBY5
To: /content/y_train.csv
100% 23.4k/23.4k [00:00<00:00, 22.4MB/s]


In [93]:
import pandas as pd
import numpy as np

In [149]:
X_test = pd.read_csv('/content/X_test.csv', encoding="euc-kr")
X_train = pd.read_csv('/content/X_train.csv', encoding="euc-kr")
y_train = pd.read_csv('/content/y_train.csv', encoding="euc-kr")

---------

# EDA

In [150]:
X_train.head()

,cust_id,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기
0,0,68282840,11264000,6860000.0,기타,강남점,19,3.894737,0.527027,17
1,1,2136000,2136000,300000.0,스포츠,잠실점,2,1.500000,0.000000,1
2,2,3197000,1639000,NaN,남성 캐주얼,관악점,2,2.000000,0.000000,1
3,3,16077620,4935000,NaN,기타,광주점,18,2.444444,0.318182,16
4,4,29050000,24000000,NaN,보석,본 점,2,1.500000,0.000000,85


In [151]:
X_train.describe()

,cust_id,총구매액,최대구매액,환불금액,내점일수,내점당구매건수,주말방문비율,구매주기
count,3500.000000,3.500000e+03,3.500000e+03,1.205000e+03,3500.000000,3500.000000,3500.000000,3500.000000
mean,1749.500000,9.191925e+07,1.966424e+07,2.407822e+07,19.253714,2.834963,0.307246,20.958286
std,1010.507298,1.635065e+08,3.199235e+07,4.746453e+07,27.174942,1.912368,0.289752,24.748682
min,0.000000,-5.242152e+07,-2.992000e+06,5.600000e+03,1.000000,1.000000,0.000000,0.000000
25%,874.750000,4.747050e+06,2.875000e+06,2.259000e+06,2.000000,1.666667,0.027291,4.000000
50%,1749.500000,2.822270e+07,9.837000e+06,7.392000e+06,8.000000,2.333333,0.256410,13.000000
75%,2624.250000,1.065079e+08,2.296250e+07,2.412000e+07,25.000000,3.375000,0.448980,28.000000
max,3499.000000,2.323180e+09,7.066290e+08,5.637530e+08,285.000000,22.083333,1.000000,166.000000


In [152]:
X_train.corr()

,cust_id,총구매액,최대구매액,환불금액,내점일수,내점당구매건수,주말방문비율,구매주기
cust_id,1.000000,-0.008004,-0.005883,0.070135,-0.025234,-0.004840,0.001900,0.020070
총구매액,-0.008004,1.000000,0.700080,0.419734,0.659084,0.090022,0.014396,-0.212944
최대구매액,-0.005883,0.700080,1.000000,0.410562,0.374147,0.018980,0.022277,-0.115837
환불금액,0.070135,0.419734,0.410562,1.000000,0.270290,-0.063114,-0.062397,-0.211125
내점일수,-0.025234,0.659084,0.374147,0.270290,1.000000,0.225264,-0.010325,-0.293200
내점당구매건수,-0.004840,0.090022,0.018980,-0.063114,0.225264,1.000000,0.007659,-0.091151
주말방문비율,0.001900,0.014396,0.022277,-0.062397,-0.010325,0.007659,1.000000,0.003372
구매주기,0.020070,-0.212944,-0.115837,-0.211125,-0.293200,-0.091151,0.003372,1.000000


서로 상관관계가 높은 변수쌍은 보이지 않는다.

In [153]:
X_train['주구매상품'].value_counts()

기타        595
가공식품      546
농산물       339
화장품       264
시티웨어      213
디자이너      193
수산품       153
캐주얼       101
명품        100
섬유잡화       98
골프         82
스포츠        69
일용잡화       64
모피/피혁      57
육류         57
남성 캐주얼     55
구두         54
건강식품       47
차/커피       44
피혁잡화       40
아동         40
축산가공       35
주방용품       32
셔츠         30
젓갈/반찬      29
주방가전       26
트래디셔널      23
남성정장       22
생활잡화       15
주류         14
가구         10
커리어         9
대형가전        8
란제리/내의      8
식기          7
액세서리        5
침구/수예       4
통신/컴퓨터      3
보석          3
남성 트랜디      2
소형가전        2
악기          2
Name: 주구매상품, dtype: int64

In [154]:
X_train['주구매지점'].value_counts()

본  점     1077
잠실점       474
분당점       436
부산본점      245
영등포점      241
일산점       198
강남점       145
광주점       114
노원점        90
청량리점       86
대전점        70
미아점        69
부평점        57
동래점        49
관악점        46
인천점        34
안양점        29
포항점        11
대구점         7
센텀시티점       6
울산점         6
전주점         5
창원점         4
상인점         1
Name: 주구매지점, dtype: int64

주구매상품과 주구매지점의 카테고리가 매우 다양하다.   
따라서 라벨 인코딩 진행 필요.   
구매 지점 자체는 지역 성균형이 불균형 한 것이 아닌 이상 크게 영향을 미치지 않을 것 같으므로 삭제하도록 한다.

In [155]:
X_train.isna().sum()

cust_id       0
총구매액          0
최대구매액         0
환불금액       2295
주구매상품         0
주구매지점         0
내점일수          0
내점당구매건수       0
주말방문비율        0
구매주기          0
dtype: int64

환불 금액이 없는 칸은 환불 기록이 없는 것이므로 0으로 입력한다.

In [156]:
y_train.head()

,cust_id,gender
0,0,0
1,1,0
2,2,1
3,3,1
4,4,0


In [157]:
y_train.isna().sum()

cust_id    0
gender     0
dtype: int64

혹시나 결측치가 있을까 싶었지만 역시 없다.

In [158]:
X_test

,cust_id,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기
0,3500,70900400,22000000,4050000.0,골프,부산본점,13,1.461538,0.789474,26
1,3501,310533100,38558000,48034700.0,농산물,잠실점,90,2.433333,0.369863,3
2,3502,305264140,14825000,30521000.0,가공식품,본 점,101,14.623762,0.083277,3
3,3503,7594080,5225000,NaN,주방용품,부산본점,5,2.000000,0.000000,47
4,3504,1795790,1411200,NaN,수산품,청량리점,3,2.666667,0.125000,8
...,...,...,...,...,...,...,...,...,...,...
2477,5977,82581500,23976000,NaN,골프,부산본점,8,1.750000,0.642857,40
2478,5978,480000,480000,NaN,섬유잡화,광주점,1,1.000000,0.000000,0
2479,5979,260003790,25750000,NaN,남성 캐주얼,본 점,19,3.736842,0.915493,18
2480,5980,88991520,18120000,NaN,육류,본 점,5,3.600000,0.444444,60


X_train과 같은 형태의 데이터로 보인다.   
X_trian에 진행할 전처리를 똑같이 진행해주도록 한다.

-------

# 전처리
cust_id와 주구매지점을 삭제한다.   
X_test의 cust_id는 결과값에 첨부해야하므로 따로 저장해둔다.

In [159]:
X_train = X_train.drop(columns='cust_id', axis=1)
X_train = X_train.drop(columns='주구매지점', axis=1)
test_cust_id = X_test['cust_id']
X_test = X_test.drop(columns='cust_id', axis=1)
X_test = X_test.drop(columns='주구매지점', axis=1)

환불금액의 결측치에 0 채우기

In [160]:
X_train['환불금액'] = X_train['환불금액'].fillna(0)
X_test['환불금액'] = X_test['환불금액'].fillna(0)

LabelEncoder를 활용하여 주구매상품 변형

In [161]:
import sklearn.preprocessing
LE = sklearn.preprocessing.LabelEncoder()

In [162]:
X_train['주구매상품'] = LE.fit_transform(X_train['주구매상품'])
X_test['주구매상품'] = LE.transform(X_test['주구매상품'])

--------

# 모델링
분류 문제이고 LabelEncoder에 의해 선형 분석은 원치 않는 영향을 받을 수 있는 관계로 RandomForestClassifier 사용

In [163]:
from sklearn.ensemble import RandomForestClassifier

In [164]:
model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=2022)
model.fit(X_train, y_train['gender'])
print(model.score(X_train, y_train['gender']))
predictions = model.predict_proba(X_test)

0.6837142857142857


68%의 성능 확인

----------

# 결과 출력

In [165]:
predictions[:,1]

array([0.4372018 , 0.18370793, 0.2032744 , ..., 0.47526614, 0.36654016,
       0.54642443])

In [166]:
output = pd.DataFrame({'cust_id' : test_cust_id, 'gender' : predictions[:,1]})

In [167]:
output

,cust_id,gender
0,3500,0.437202
1,3501,0.183708
2,3502,0.203274
3,3503,0.420929
4,3504,0.481358
...,...,...
2477,5977,0.456722
2478,5978,0.516874
2479,5979,0.475266
2480,5980,0.366540


csv 파일로 저장

In [168]:
output.to_csv("20220525.csv", index=False)

In [169]:
data = pd.read_csv('/content/20220525.csv')

In [170]:
data

,cust_id,gender
0,3500,0.437202
1,3501,0.183708
2,3502,0.203274
3,3503,0.420929
4,3504,0.481358
...,...,...
2477,5977,0.456722
2478,5978,0.516874
2479,5979,0.475266
2480,5980,0.366540


csv 파일이 정상적으로 생성됨을 확인